In [ ]:
import pandas as pd
import yfinance as yf
import optuna
import numpy as np


selected_tickers = ['NVDA', 'AAPL', 'AMZN', 'META', 'GOOGL'] #based on Optuna scores and discretion

def get_annualized_return(ticker, start_date, end_date):

    data = yf.download(ticker, start=start_date, end=end_date, progress=False)

    if 'Adj Close' in data.columns:
        prices = data['Adj Close']
    else:
        prices = data['Close']
    returns = prices.pct_change().dropna()

    annualized_return = float((returns.mean() * 252).iloc[0] if hasattr(returns.mean(), 'iloc') else returns.mean() * 252)
    return annualized_return

def objective(trial):

    lookback_years = trial.suggest_int("lookback_years", 3, 10)

    d
    training_end = "2021-12-31"
    training_start_year = 2021 - lookback_years + 1  # e.g., for a 5-year lookback, start in 2017
    training_start = f"{training_start_year}-01-01"

    # Validation period: the year 2022
    validation_start = "2022-01-01"
    validation_end = "2022-12-31"

    errors = []
    for ticker in selected_tickers:
        try:

            train_ret = get_annualized_return(ticker, training_start, training_end)

            val_ret = get_annualized_return(ticker, validation_start, validation_end)

            errors.append(abs(train_ret - val_ret))
        except Exception as e:

            continue

    avg_error = np.mean(errors) if errors else np.inf
    return avg_error


study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=1000)

print("Best lookback period (years):", study.best_params["lookback_years"])


In [ ]:
selected_tickers = ['NVDA', 'AAPL', 'AMZN', 'META', 'GOOGL']
start_date = "2012-01-01"
end_date   = "2022-01-01"


data = yf.download(selected_tickers, start=start_date, end=end_date, progress=False)
if isinstance(data.columns, pd.MultiIndex):
    if "Adj Close" in data.columns.get_level_values(0):
        stock_prices = data["Adj Close"]
    else:
        stock_prices = data["Close"]
else:
    if "Adj Close" in data.columns:
        stock_prices = data["Adj Close"]
    else:
        stock_prices = data["Close"]


stock_returns = stock_prices.pct_change().dropna()


ff = web.DataReader("F-F_Research_Data_Factors", "famafrench", start=start_date, end=end_date)[0]

example_index = ff.index[0]
if '-' in str(example_index):
    ff.index = pd.to_datetime(ff.index.astype(str), format="%Y-%m")
else:
    ff.index = pd.to_datetime(ff.index.astype(str), format="%Y%m")

ff_daily = ff.resample('D').ffill()

ff_daily = ff_daily.loc[ff_daily.index >= pd.to_datetime(start_date)]

ff_daily.columns = ff_daily.columns.str.replace('-', '_')

ff_daily[['Mkt_RF', 'SMB', 'HML', 'RF']] = ff_daily[['Mkt_RF', 'SMB', 'HML', 'RF']] / 100



expected_returns = {}
for ticker in selected_tickers:
    df = pd.concat([stock_returns[ticker], ff_daily], axis=1).dropna()
    df.columns = ['stock_return', 'Mkt_RF', 'SMB', 'HML', 'RF']
    df['excess_return'] = df['stock_return'] - df['RF']

    X = df[['Mkt_RF', 'SMB', 'HML']]
    X = sm.add_constant(X)
    y = df['excess_return']
    model = sm.OLS(y, X).fit()


    avg_factors = ff_daily[['Mkt_RF', 'SMB', 'HML']].mean()
    expected_daily_excess = (model.params['const'] +
                             model.params['Mkt_RF'] * avg_factors['Mkt_RF'] +
                             model.params['SMB'] * avg_factors['SMB'] +
                             model.params['HML'] * avg_factors['HML'])
    annual_expected_excess = expected_daily_excess * 252
    annual_rf = ff_daily['RF'].mean() * 252
    expected_returns[ticker] = annual_rf + annual_expected_excess



cov_matrix = stock_returns[selected_tickers].cov() * 252


num_portfolios = 15000
results = np.zeros((3, num_portfolios))
weights_record = []
multi_expected_returns = np.array([expected_returns[ticker] for ticker in selected_tickers])
risk_free_rate_annual = ff_daily['RF'].mean() * 252

n = len(selected_tickers)
min_weight = 0.05

for i in range(num_portfolios):

    weights = min_weight + np.random.dirichlet(np.ones(n)) * (1 - n * min_weight)
    weights_record.append(weights)

    port_return = np.dot(weights, multi_expected_returns)
    port_vol = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    sharpe_ratio = (port_return - risk_free_rate_annual) / port_vol

    results[0, i] = port_return
    results[1, i] = port_vol
    results[2, i] = sharpe_ratio

results_df = pd.DataFrame(results.T, columns=['Return', 'Volatility', 'Sharpe Ratio'])
max_sharpe_idx = results_df['Sharpe Ratio'].idxmax()
max_sharpe_portfolio = results_df.iloc[max_sharpe_idx]
optimal_weights = weights_record[max_sharpe_idx]

print("\nOptimal Portfolio using Multifactor Expected Returns (MPT):")
print(max_sharpe_portfolio)
optimal_weights_df = pd.DataFrame({'Ticker': selected_tickers, 'Weight (%)': optimal_weights*100})
print("\nOptimal Weights (each >= 5%):")
print(optimal_weights_df)

In [ ]:
import pandas as pd
from plotnine import (
    ggplot, aes, geom_point, geom_label, scale_color_gradientn,
    labs, theme_minimal, theme, element_text, element_blank
)

optimal_df = pd.DataFrame({
    'Volatility': [max_sharpe_portfolio['Volatility']],
    'Return': [max_sharpe_portfolio['Return']],
    'Sharpe Ratio': [max_sharpe_portfolio['Sharpe Ratio']]
})

vol_pct = 100 * max_sharpe_portfolio['Volatility']
ret_pct = 100 * max_sharpe_portfolio['Return']
sharpe_val = max_sharpe_portfolio['Sharpe Ratio']

optimal_label = (f"Vol: {vol_pct:.1f}%\n"
                 f"Ret: {ret_pct:.1f}%\n"
                 f"Sharpe: {sharpe_val:.2f}")
optimal_df['Label'] = optimal_label

custom_colors = [
    "#0096c7",
    "#0077b6",
    "#023e8a"
]

gg = (
    ggplot(results_df, aes(x='Volatility', y='Return', color='Sharpe Ratio'))
    + geom_point(size=2.5, alpha=0.7)
    + geom_point(
        mapping=aes(x='Volatility', y='Return'),
        data=optimal_df,
        shape='^',
        color='black',
        size=6
    )
    + geom_label(
        mapping=aes(label='Label'),
        data=optimal_df,
        color='white',
        fill='black',
        size=10,
        label_padding=0.25,
        label_size=0,

        nudge_x=0.015,
        nudge_y=0.015
    )
    + scale_color_gradientn(colors=custom_colors)
    + labs(
        title="Efficient Frontier using Multifactor Expected Returns",
        x="Annualized Volatility",
        y="Annualized Return",
        color="Sharpe Ratio"
    )
    + theme_minimal(base_size=12, base_family='sans')
    + theme(
        figure_size=(12, 6),
        axis_title=element_text(size=14),
        plot_title=element_text(size=16),
        legend_position='right',
        panel_grid_minor=element_blank()
    )
)

gg


In [ ]:
def block_bootstrap_daily_returns(returns_data, weights, days_to_simulate, block_size, num_simulations=1000):
    T = len(returns_data)
    returns_array = returns_data.values
    n_stocks = len(weights)

    daily_returns_all = np.zeros((num_simulations, days_to_simulate), dtype=np.float64)

    for sim_idx in range(num_simulations):
        path_daily_returns = []
        num_blocks = days_to_simulate // block_size
        leftover_days = days_to_simulate % block_size


        for _ in range(num_blocks):
            start_idx = np.random.randint(0, T - block_size)
            block = returns_array[start_idx:start_idx+block_size, :]  )
            for day_returns in block:
                path_daily_returns.append(np.dot(weights, day_returns))


        if leftover_days > 0:
            start_idx = np.random.randint(0, T - leftover_days)
            leftover_block = returns_array[start_idx:start_idx+leftover_days, :]
            for day_returns in leftover_block:
                path_daily_returns.append(np.dot(weights, day_returns))

        daily_returns_all[sim_idx, :] = path_daily_returns

    return daily_returns_all


def objective(trial):


    block_size = trial.suggest_int('block_size', 5, 60)


    daily_returns_all = block_bootstrap_daily_returns(
        returns_data=stock_returns[selected_tickers].dropna(),
        weights=optimal_weights,
        days_to_simulate=252,
        block_size=block_size,
        num_simulations=10000
    )

    n_sims = daily_returns_all.shape[0]
    sharpe_ratios = np.zeros(n_sims)


    daily_rf = 0.0125 / 252

    for i in range(n_sims):
        path = daily_returns_all[i, :]
        mean_path = np.mean(path)
        std_path = np.std(path)


        excess_mean = mean_path - daily_rf

        if std_path == 0:
            sharpe_ratios[i] = -9999
        else:

            sharpe_ratios[i] = (excess_mean / std_path) * np.sqrt(252)

    avg_sharpe = np.mean(sharpe_ratios)
    return avg_sharpe


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10000, n_jobs=-1)

print("Best parameters:", study.best_params)
print("Best daily Sharpe (with 1.25% RF):", study.best_value)


In [ ]:
best_block_size = study.best_params['block_size']
final_daily_returns = block_bootstrap_daily_returns(
    returns_data=stock_returns[selected_tickers].dropna(),
    weights=optimal_weights,
    days_to_simulate=252,
    block_size=best_block_size,
    num_simulations=10000
)


In [ ]:
import pandas as pd
import numpy as np
from plotnine import (
    ggplot, aes, geom_histogram, geom_label, labs,
    theme_minimal, theme, element_text
)


final_annual_returns = (1 + final_daily_returns).prod(axis=1) - 1


mean_ret = final_annual_returns.mean()
std_ret  = final_annual_returns.std()
excess_mean = mean_ret - 0.0125
if std_ret != 0:
    final_sharpe = (excess_mean / std_ret) * np.sqrt(252)
else:
    final_sharpe = 0

pct_5  = np.percentile(final_annual_returns, 5)
pct_95 = np.percentile(final_annual_returns, 95)


df = pd.DataFrame({'Annual Return': final_annual_returns})

annotation_text = (
    f"Mean Annual Return: {mean_ret*100:.2f}%\n"
    f"Annual Volatility: {std_ret*100:.2f}%\n"
    f"Sharpe Ratio (1.25% RF): {final_sharpe:.2f}\n"
    f"5th Percentile (VaR): {pct_5*100:.2f}%\n"
    f"95th Percentile: {pct_95*100:.2f}%"
)


hist_gg = (
    ggplot(df, aes(x='Annual Return'))
    + geom_histogram(bins=50, fill='skyblue', color='black', alpha=0.7)
    + geom_label(
        inherit_aes=False,
        x=1.0,
        y=700,
        label=annotation_text,
        label_size=0,
        size=10,
        color='black',
        fill='white'
    )
    + labs(
        title="Distribution of Final Annual Returns",
        x="Annual Return",
        y="Frequency"
    )
    + theme_minimal(base_size=12)
    + theme(
        figure_size=(10, 6),
        plot_title=element_text(size=16, weight='bold'),
        axis_title=element_text(size=14)
    )
)

hist_gg
